# EECS 338 
## Webscraping Notebook

### 04/24/2019
### This is a jupyter notebook to be used to scrape internet data from forevermissed.com

In [ ]:
from __future__ import with_statement
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
browser = webdriver.Chrome('/Users/aea/Downloads/chromedriver')
names = ['Smith', 'Johnson', 'Jones', 'Moen', 'Chen', 'Alwan', 'Briggs', 'Louis',
         "Williams", "Brown", "Garcia", "Rodriguez", "Miller", "Davis"]
# names = ['Moen', 'Smith']

In [ ]:
links = []  # create empty list for all links

for name in names:
    element_names = []    # create temporary list for element names
    
    # perform search by last name
    browser.get('https://www.forevermissed.com/findmemorial/?q=' + name)
    
    # obtain search result line
    search_result = browser.find_elements_by_class_name("bar")
    
    # obtain number of people that match search result
    for res in search_result:
        # convert to string type
        num_results = str(res.text)

        # obtain the number of search results found
        num_results = ''.join(filter(lambda x: x.isdigit(), num_results))
        
    # we have the number....
    # num_results is the number of search results for the last name
    num_results = int(num_results)    
    offset = 0

    # while loop that resets webpage to display 30 new individuals so that we can obtain 30 more links
    while offset < num_results:
        
        # navigate to webpage with 30 search results
        browser.get('https://www.forevermissed.com/findmemorial/?q=' + name + '&offset=' + str(offset))
        
        # increase the offset by 30 to obtain 30 new results next loop through
        offset += 30
    
        # obtain the link for the individual
        inputElement = browser.find_elements_by_class_name("memorial-link")

        # append the links to a list
        for el in inputElement:
            element_names.append(el.text)

    # append the list of links to a larger list        
    links.append(element_names)
    
# print(links)

# create a new list of "valid" links (links that will not be too long)
valid_links = []

for i in range(0, len(links)):
    for link in links[i]:
        if len(str(link)) <= 47:
            valid_links.append(link)
            
print(valid_links)

# output links to a text file
with open('memorial_links.txt', 'w') as f:
    for _list in valid_links:
        f.write(str(_list) + '\n')          
f.close()


In [ ]:
### unable to retrieve only "user_text" text, so we will have to take out all "txt user_text" and "body user_text" text from the list...


# (possible) next steps:    
##   1) Implement 4 dictionaries:
##     a) one will have the name of the person and the link
##     b) one will have the name and tribute text of the person 
##     c) one will have the name and story text of the person
##     d) one will have the name and age of person


In [ ]:
valid_links = open('memorial_links.txt', "r").readlines()
valid_links = map(lambda s: s.strip(), valid_links)
valid_links = list(valid_links)
len(valid_links)

In [ ]:
### test

from bs4 import BeautifulSoup as bs
import requests
import time

link = 'james-albert-smith-jr.forevermissed.com'
stuff = 'http://' + link + '/stories'
response = requests.get('http://' + link + '/stories')
# print(response.text) 

def has_six_characters(css_class):
    return css_class is not None and len(css_class) == 9

soup = bs(response.text, "html.parser")
# print(soup.prettify())
            
# obtain the text from written story (class = "user_text")
user_text = soup.find_all(class_="user_text")

print(stuff)
print(user_text)

In [ ]:
valid_links[4802]

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import time

# collect tribute text from the about tab (the home page)
# tribute_text = []
# tribute_text2 = []
# v_links = valid_links[4803:len(valid_links)]
# i = 4802

# for link in v_links:
#     i += 1
    
#     response = ''
    
#     while response == '':
#         try:   
#             # navigate to the home page of the respective url
#             response = requests.get('http://' + link)
#             # print(response.text)  

#             soup = bs(response.text, "html.parser")
#             # print(soup.prettify())

#             # obtain the text from written tribute (class = "txt user_text")
#             user_text = soup.findAll(attrs={'class':'txt user_text'})

#             # iterate through each instance and convert text to string type
#             for text in user_text:
#                 tribute_text2.append(str(text))
                
#             print(i)
            
#             break
            
#         except:
#             # implement a 5 second pause
#             time.sleep(5)
#             continue


# collect text from the stories tab 
story_text = []
i = 0

# for link in valid_links:
for link in valid_links:
    i += 1
    
    response = ''
    
    while response == '':  
        try:   
            # navigate to stories tab by adjusting the url
            response = requests.get('http://' + link + '/stories')
            # print(response.text)  

            soup = bs(response.text, "html.parser")
            # print(soup.prettify())
            
            # obtain the text from written story (class = "user_text")
            user_text = soup.find_all(class_= "user_text")
#             print(user_text)

            # iterate through each instance and convert text to string type and append to new list
            for text in user_text:
                story_text.append(str(text))   

            print(i)
            
            break
            
        except:
            # implement a 5 second pause
            time.sleep(5)
            
            continue

In [ ]:
# output to different text files
with open('forevermissed_tribute_text.txt', 'w') as f:
    for item in tribute_text:
        f.write(str(item) + '\n')            
f.close()


with open('forevermissed_story_text.txt', 'w') as f:
    for item in story_text:
        f.write(str(item) + '\n')           
f.close()

In [ ]:
len(story_text)


In [ ]:
import re

tribute_text1 = tribute_text[:len(tribute_text)//2]
tribute_text2 = tribute_text[len(tribute_text)//2:]

cleaned_story_text = []
cleaned_tribute_text= []

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

# for item in story_text:
#     item = remove_tags(str(item))
#     item = item.replace('\xa0', ' ')
#     cleaned_story_text.append(item)

# print(cleaned_story_text)

for item in tribute_text1:
    item = remove_tags(str(item))
    item = item.replace('\n', ' ')
    cleaned_tribute_text.append(item)
    
for item in tribute_text2:
    item = remove_tags(str(item))
    item = item.replace('\n', ' ')
    cleaned_tribute_text.append(item)
    
print(cleaned_tribute_text)


# output to different text files
# with open('forevermissed_tribute_text.txt', 'w') as f:
#     for item in cleaned_tribute_text:
#         f.write(str(item) + '\n')            
# f.close()


# with open('forevermissed_story_text.txt', 'w') as f:
#     for item in cleaned_story_text:
#         f.write(str(item) + '\n')           
# f.close()

In [ ]:
# This part of the script is used to clean the data sets from unnecessary characters

cleaned_tribute_text = []
cleaned_story_text = []

# clean the tribute data
for item in tribute_text:
    
    # strip the text of the unecessary elements
    item = item.strip('\n')
    item = item.strip('<div class="txt user_text">')
    item = item.strip('</')
    item = item.replace('\n', ' ')
    
    cleaned_tribute_text.append(item)
    

# clean the story data
for item in story_text:

    # strip the text of the unecessary elements
    item = item.strip('\n')
    item = item.strip('<div class="txt user_text">')
    item = item.strip('</')
    item = item.replace('\n', ' ')
    
    cleaned_story_text.append(item)
    
for i in cleaned_story_text:
    print(i)




In [ ]:
str_list = list(filter(None, str_list)) # fastest